# Interactive Erasus Demo

Run a quick unlearning experiment with different strategies and selectors. Adjust the parameters below and run all cells.

In [ ]:
# Parameters (edit and re-run)
STRATEGY = "gradient_ascent"   # or scrub, fisher_forgetting, negative_gradient
SELECTOR = "random"            # or gradient_norm, influence, full
EPOCHS = 3
PRUNE_RATIO = 0.2
LR = 1e-3

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from erasus.unlearners import ErasusUnlearner
from erasus.metrics.metric_suite import MetricSuite
import erasus.strategies
import erasus.selectors

device = "cuda" if torch.cuda.is_available() else "cpu"
model = nn.Sequential(nn.Linear(32, 64), nn.ReLU(), nn.Linear(64, 5)).to(device)
forget_loader = DataLoader(TensorDataset(torch.randn(60, 32), torch.randint(0, 5, (60,))), batch_size=16, shuffle=True)
retain_loader = DataLoader(TensorDataset(torch.randn(200, 32), torch.randint(0, 5, (200,))), batch_size=16, shuffle=True)

unlearner = ErasusUnlearner(
    model=model,
    strategy=STRATEGY,
    selector=SELECTOR,
    device=device,
    selector_kwargs={"prune_ratio": PRUNE_RATIO},
    strategy_kwargs={"lr": LR},
)
result = unlearner.fit(forget_data=forget_loader, retain_data=retain_loader, epochs=EPOCHS)
metrics = MetricSuite(["accuracy"]).run(unlearner.model, forget_loader, retain_loader)
print("Strategy:", STRATEGY, "| Selector:", SELECTOR)
print("Forget loss:", result.forget_loss_history)
print("Metrics:", {k: round(v, 4) for k, v in metrics.items() if k != "_meta" and isinstance(v, (int, float))})